In [1]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [2]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new1.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2010t6+.csv', index=False)
df

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797868,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797869,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797870,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797871,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [ ]:
data = pd.read_csv('общие_данные_2010t6+.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]
protests = data[data['Event Text'].isin(protest_events)]
protests.to_csv('протесты_2010t6+.csv', index=False)


protests1 = protests[protests['year'].isin([2011, 2012, 2013, 2014])].reset_index(drop=True)
protests1 = protests1.drop_duplicates(subset=['oktmo']).reset_index(drop=True)
protests1.to_csv('протесты2009_2013_2010t6+.csv', index=False)
protests1

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2011,Тверская,Андреапольский,28602000,20.4,0.00,0.0,400.0,13664.0,0.0,...,56587.0,9.4,19.0,124.0,58572.12,13242.0,642.98,13205.0,305086.0,Criticize or denounce
1,2011,Архангельская,Архангельск,11701000,538.2,2.94,2400.0,5900.0,355623.0,-0.7,...,-565298.0,11.1,11.8,3957.0,29587.00,356073.0,417.00,27871.7,29445.0,Criticize or denounce
2,2011,Башкортостан,Аургазинский,80605000,2.1,12.00,0.0,30100.0,36882.0,0.0,...,13033.0,13.0,16.0,473.0,2239.00,36555.0,426.00,12294.5,201403.0,Criticize or denounce
3,2011,Кабардино-Балкарская,Баксанский,83610000,62.6,0.00,1500.0,81050.0,61054.0,10.4,...,112764.0,20.1,9.7,1227.0,1325.82,61147.0,225.00,9631.4,75350.0,Criticize or denounce
4,2011,Башкортостан,Благовещенск,80615101,0.0,0.00,0.0,4000.0,34286.0,0.0,...,4463.0,0.0,0.0,499.0,0.00,0.0,0.00,0.0,6466.0,Demonstrate or rally
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2014,Краснодарский,Тимашевский,3653000,22.0,93.92,470.0,5657.0,109292.0,-0.5,...,-15580.0,13.7,14.2,1503.0,16100.30,109777.0,0.00,26709.6,150643.0,Demonstrate or rally
171,2014,Томская,Томский,69654000,29.7,80.49,3073.0,12397.0,71541.0,2.8,...,-70581.0,15.1,12.3,1081.0,22974.00,71647.0,0.00,27653.1,1003885.0,Demonstrate or rally
172,2014,Приморский,Хасанский,5648000,23.5,38.13,1366.0,900.0,33167.0,0.3,...,-36475.2,14.0,13.7,459.0,133670.30,32854.0,0.00,34037.9,413003.0,Criticize or denounce
173,2014,Башкортостан,Чекмагушевский,80656000,10.0,49.20,0.0,1200.0,29559.0,-3.1,...,11694.0,15.0,18.0,426.0,23892.00,29316.0,0.00,18149.5,168616.0,Demonstrate or rally


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [ ]:
lol = pd.read_csv("протесты_2010t6+.csv") 
sobytia = pd.read_csv("общие_данные_2010t6+.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t6+.csv', index=False)
noprotestALL

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623222,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623223,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623224,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623225,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [ ]:
lol = pd.read_csv("протесты2009_2013_2010t6+.csv") 
sobytia = pd.read_csv("общие_данные_2010t6+.csv")

df_2011 = sobytia[sobytia['year'] == 2010]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2008_2010t6+.csv', index=False)
final_df



,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2010,Карачаево-Черкесская,Абазинский,91601000,0.00,0.00,0.0,3800.0,13280.0,0.0,...,-1356.2,11.1,9.1,191.0,1447.9,17175.0,230.00,10943.0,30019.0,NaN
1,2010,Удмуртская,Алнашский,94602000,22.40,0.00,0.0,660.0,20718.0,0.0,...,3398.0,18.9,16.4,387.0,6349.1,20428.0,360.00,8804.1,89600.0,Investigate
2,2010,Мурманская,Апатиты,47705000,1.40,0.00,300.0,500.0,60003.0,-4.2,...,52509.0,10.9,15.1,655.0,0.0,59826.0,510.80,25817.6,247136.0,NaN
3,2010,Башкортостан,Архангельский,80603000,6.20,0.00,0.0,0.0,19829.0,0.0,...,8468.0,14.6,17.3,271.0,1610.0,18503.0,483.00,11085.9,242145.0,NaN
4,2010,Башкортостан,Архангельский,80603410,0.00,0.00,0.0,0.0,0.0,0.0,...,-292.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6347.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2010,Калмыкия,Элиста,85701000,51.30,0.00,0.0,2963.0,106966.0,0.0,...,-86985.4,16.2,10.1,1775.0,11329.0,109647.0,323.00,14293.4,39657.0,Reject
119,2010,Сахалинская,Южно-Сахалинск,64701000,171.00,1.03,0.0,4700.0,0.0,0.0,...,-427701.0,12.1,11.4,2286.0,18556.0,189069.0,389.08,43408.3,89827.0,Consult
120,2010,Саха,Якутск,98701000,311.76,0.00,1800.0,1300.0,278691.0,0.0,...,1380480.0,19.0,8.3,5426.0,0.0,285106.0,319.00,32882.1,355334.0,"Arrest, detain, or charge with legal action"
121,2010,Башкортостан,Янаульский,80659000,10.70,0.00,200.0,5000.0,48916.0,0.0,...,2967.0,14.3,14.9,690.0,564.0,48213.0,432.00,17995.5,209381.0,Make statement


Создание регрессии

In [ ]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)


df_protests = pd.read_csv("протесты2009_2013_2010t6+.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t6+.csv")
df_events_full = pd.read_csv("общие_данные_2010t6+.csv")

required_years = {2010.0, 2015.0, 2016.0, 2017.0}

features = [
            # 'Население', 
            'ПотреблениеЭ', 
            'ПотреблениеВ', 
            'Бюджет',
            'БезДорог', 
            'Земля',

            # 'Прибыль',
            'Квартиры',
            # 'Зарплата',
            # 'Расходы',

            # 'СреднегодН',
            # 'ПотреблениеТ', 
            # 'ЭконАктив'
            # 'Прирост', 
            # 'Рождаемость',
            # 'Смертность',
            # 'Родившихся',
            # 'Инвестиции',
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)


exclude_dict = {
 'Абазинский': ['Хабезский', 'Усть-Джегутинский', 'Черкесский', 'Прикубанский', 'Адыге-Хабльский'],
 'Алнашский': ['Граховский', 'Менделеевский', 'Агрызский', 'Можгинский'],
 'Апатиты': ['Полярные Зори', 'Кандалакшский', 'Терский', 'Кировск', 'Оленегорск', 'Мончегорск'],
 'Архангельский': ['Гафурийский', 'Белорецкий', 'Ашинский', 'Иглинский', 'Кармаскалинский'],
 'Асекеевский': ['Грачёвский', 'Матвеевский', 'Абдулинский', 'Бугурусланский', 'Бузулукский'],
 'Аургазинский': ['Кармаскалинский', 'Давлекановский', 'Альшеевский', 'Стерлитамакский', 'Гафурийский'],
 'Баксан': ['Баксанский', 'Чегемский', 'Прохладненский'],
 'Баксанский': ['Баксан', 'Чегемский', 'Прохладненский', 'Зольский', 'Эльбрусский', 'Кировский'],
 'Бердск': ['Новосибирский', 'Искитим', 'Искитимский', 'Кольцово'],
 'Березники': ['Соликамский', 'Александровский', 'Добрянский', 'Юсьвинский', 'Кудымкарский', 'Косинский'],
 'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
 'Буйнакск': ['Буйнакский'],
 'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
 'Воловский': ['Тепло-Огаревский', 'Каменский', 'Ефремов', 'Куркинский', 'Богородицкий'],
 'Волосовский': ['Кингисеппский', 'Сланцевский', 'Лужский', 'Гатчинский', 'Ломоносовский'],
 'Даниловский': ['Котовский', 'Руднянский', 'Еланский', 'Михайловка', 'Фроловский', 'Ольховский'],
 'Дербент': ['Дербентский', 'Докузпаринский', 'Курахский', 'Дагестанские Огни'],
 'Дмитриевский': ['Хомутовский', 'Севский', 'Комаричский', 'Дмитровский', 'Железногорский', 'Конышевский'],
 'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
 'Домбаровский': ['Ясненский', 'Комаровский', 'Новоорский', 'Орск'],
 'Жуковский': ['Раменский', 'Лыткарино', 'Люберцы', 'Бронницы'],
 'Залегощенский': ['Орёл', 'Орловский', 'Свердловский', 'Покровский', 'Верховский', 'Новосильский', 'Мценский'],
 'Заполярный': ['Мезенский', 'Усть-Цилемский', 'Усинск', 'Инта', 'Воркута'],
 'Избербаш': ['Каякентский', 'Сергокалинский', 'Дахадаевский', 'Карабудахкентский'],
 'Кашарский': ['Миллеровский', 'Чертковский', 'Верхнедонской', 'Боковский', 'Советский', 'Милютинский', 'Тарасовский'],
 'Кизильский': ['Карталинский', 'Брединский', 'Кваркенский', 'Баймакский', 'Сибай', 'Абзелиловский', 'Агаповский'],
 'Кизляр': ['Шелковской', 'Кизлярский', 'Тарумовский', 'Бабаюртовский', 'Цунтинский', 'Тляратинский', 'Гумбетовский'],
 'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
 'Кинель': ['Волжский', 'Кинельский'],
 'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
 'Кировский': ['Куйбышевский', 'Дятьковский', 'Людиновский', 'Думиничский', 'Сухиничский', 'Барятинский', 'Спас-Деменский'],
 'Кисловодск': ['Предгорный', 'Малокарачаевский', 'Зольский'],
 'Комаричский': ['Суземский', 'Севский', 'Дмитриевский', 'Дмитровский', 'Брасовский'],
 'Комсомольск-на-Амуре': ['Комсомольский'],
 'Комсомольский': ['Нефтегорский', 'Богатовский', 'Кинель-Черкасский', 'Красноярский', 'Кинель', 'Волжский'],
 'Крестецкий': ['Новгородский', 'Парфинский', 'Демянский', 'Валдайский', 'Окуловский', 'Маловишерский'],
 'Курумканский': ['Северо-Байкальский', 'Баунтовский', 'Баргузинский'],
 'Ленинский': ['Домодедово', 'Подольск', 'Раменский', 'Лыткарино', 'Дзержинский', 'Подольск'],
 'Лермонтов': ['Предгорный', 'Железноводск', 'Пятигорск', 'Ессентуки', 'Минераловодский'],
 'Ловозерский': ['Терский', 'Кировск', 'Оленегорск', 'Кольский'],
 'Лыткарино': ['Люберцы', 'Раменский', 'Ленинский', 'Дзержинский'],
 'Магадан': ['Ольский', 'Хасынский'],
 'Мегион': ['Нижневартовский', 'Нижневартовск', 'Лангепас'],
 'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский', 'Урдомское'],
 'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
 'Надымский': ['Пуровский', 'Сургутский', 'Белоярский', 'Приуральский', 'Ямальский', 'Тазовский', 'Новый Уренгой'],
 'Невинномысск': ['Кочубеевский'],
 'Некоузский': ['Мышкинский', 'Сонковский', 'Краснохолмский', 'Брейтовский', 'Рыбинский'],
 'Новгородский': ['Старорусский', 'Парфинский', 'Крестецкий', 'Маловишерский', 'Чудовский', 'Тосненский', 'Лужский', 'Батецкий', 'Шимский'],
 'Нягань': ['Октябрьский'],
 'Орёл': ['Орловский'],
 'Палласовский': ['Быковский', 'Николаевский', 'Старополтавский', 'Знаменск', 'Жанибекский'],
 'Прибайкальский': ['Кабанский', 'Иволгинский', 'Улан-Удэ', 'Заиграевский', 'Хоринский', 'Баргузинский', 'Усть-Джилиндинское'],
 'Псков': ['Псковский'],
 'Пятигорск': ['Предгорный', 'Ессентуки', 'Лермонтов', 'Железноводск'],
 'Ростов': ['Ростовский'],
 'Салехард': ['Приуральский', 'Шурышкарский'],
 'Саранск': ['Рузаевский', 'Лямбирский', 'Кочкуровский'],
 'Серафимовичский': ['Советский', 'Боковский', 'Шолоховский', 'Кумылженский', 'Михайловка', 'Фроловский', 'Клетский'],
 'Сергиевский': ['Красноярский', 'Елховский', 'Кошкинский', 'Челно-Вершинский', 'Шенталинский', 'Похвистневский', 'Кинель-Черкасский'],
 'Слюдянский': ['Закаменский', 'Тункинский', 'Кабанский', 'Иркутский', 'Шелеховский', 'Усольский', 'Баяндаевский'],
 'Солигаличский': ['Буйский', 'Галичский', 'Чухломский', 'Бабушкинский', 'Тотемский', 'Междуреченский', 'Грязовецкий'],
 'Сыктывдинский': ['Сыктывкар', 'Сысольский', 'Ленский', 'Усть-Вымский', 'Княжпогостский', 'Корткеросский', 'Койгородский'],
 'Тобольск': ['Тобольский'],
 'Торжокский': ['Кувшиновский', 'Старицкий', 'Калининский', 'Лихославльский', 'Спировский', 'Вышневолоцкий', 'Шепелевское'],
 'Торопецкий': ['Западнодвинский', 'Андреапольский', 'Холмский', 'Локнянский', 'Великолукский', 'Куньинский'],
 'Тюмень': ['Тюменский', 'Нижнетавдинский'],
 'Ульяновский': ['Ульяновск', 'Новоульяновск', 'Чердаклинский'],
 'Хвойнинский': ['Боровичский', 'Мошенской', 'Пестовский', 'Чагодощенский', 'Бокситогорский', 'Любытинский'],
 'Холмогорский': ['Приморский', 'Пинежский', 'Виноградовский', 'Мирный', 'Плесецкий'],
 'Чебоксарский': ['Чебоксары', 'Моргаушский', 'Красноармейский', 'Цивильский', 'Мариинско-Посадский', 'Звениговский', 'Килемарский'],
 'Чегемский': ['Нальчик', 'Черекский', 'Эльбрусский', 'Баксанский', 'Баксан', 'Урванский'],
 'Чердынский': ['Троицко-Печорский', 'Красновишерский', 'Соликамский', 'Косинский', 'Гайнский', 'Усть-Куломский'],
 'Череповец': ['Череповецкий'],
 'Шаховская': ['Можайский', 'Гагаринский', 'Зубцовский', 'Лотошинский', 'Волоколамский'],
 'Шуйский': ['Шуя', 'Савинский', 'Южский', 'Палехский', 'Родниковский', 'Ивановский', 'Лежневский'],
 'Элиста': ['Ики-Бурульский', 'Приютненский', 'Целинный'],
 'Якутск': ['Хангаласский', 'Мегино-Кангаласский', 'Горный', 'Намский', 'Усть-Алданский'],
 'Янаульский': ['Куединский', 'Татышлинский', 'Бураевский', 'Калтасинский', 'Краснокамский', 'Камбарский', 'Нефтекамск', 'Чайковский'],
 'Ярославский': ['Ярославль', 'Тутаевский', 'Большесельский', 'Борисоглебский', 'Гаврилов-Ямский', 'Некрасовский', 'Даниловский'],
  "Владивостокский": ["Хасанский", "Надеждинский", "Артемовский", "Шкотовский", "Большой Камень", "Фокино"],
    "Воротынский": ["Пильнинский", "Спасский", "Лысковский", "Воскресенский", "Юринский", "Горномарийский", "Ядринский"],
    "Джейрах": ["Итум-Калинский", "Ачхой-Мартановский", "Сунженский", "Пригородный", "Владикавказ"],
    "Кизилюртовский": ["Кумторкалинский", "Хунзахский", "Буйнакский", "Казбековский", "Хасавюртовский",
                       "Кизилюрт", "Цумадинский", "Бабаюртовский", "Ахвахский"],
    "Кичменгско-Городецкий": ["Нюксенский", "Бабушкинский", "Никольский", "Вохомский", "Опаринский", "Подосиновский", "Великоустюгский"],
    "Красногвардейский": ["Усть-Лабинский", "Шовгеновский", "Белореченский", "Теучежский", "Краснодар", "Динский"],
    "Красноярск": ["Березовский", "Дивногорск", "Емельяновский"],
    "Нестеровский": ["Краснознаменский", "Гусевский", "Озерский"],
    "Сегежский": ["Беломорский", "Муезерский", "Медвежьегорский", "Онежский"],
    "Ульяновский": ["Ульяновск", "Цильнинский", "Майнский", "Тереньгульский",
                    "Сенгилеевский", "Новоульяновск", "Чердаклинский", "Тетюшский"],
    "Хасавюртовский": ["Хасавюрт", "Новолакский", "Ахвахский", "Гудермесский", "Шелковский",
                       "Бабаюртовский", "Хунзахский", "Кизилюртовский", "Кизилюрт", "Казбековский"],
    "Чайковский": ["Еловский", "Куединский", "Янаульский", "Нефтекамск", "Камбарский",
                   "Сарапульский", "Завьяловский", "Воткинский"],
    "Шовгеновский": ["Курганинский", "Кошехабльский", "Гиагинский", "Белореченский", "Красногвардейский", "Усть-Лабинский"],
    "Южно-Сахалинск": ["Корсаковский", "Анивский", "Холмский", "Долинский"],
    "Богучанский": ["Эвенкийский", "Кежемский", "Чунский", "Тайшетский", "Абанский", "Тасеевский", "Мотыгинский"],
    "Буздякский": ["Чекмагушевский", "Шаранский", "Туймазинский", "Белебеевский", "Давлекановский", "Чишминский", "Благоварский"],
    "Гайский": ["Кувандыкский", "Хайбуллинский", "Новотроицк", "Орск", "Новоорский", "Домбаровский"],
    "Карасукский": ["Баганский", "Здвинский", "Краснозерский", "Бурлинский"],
    "Липецк": ["Грязинский", "Липецкий", "Добровский"],
    "Манский": ["Березовский", "Балахтинский", "Курагинский", "Партизанский", "Уярский"],
    "Медвенский": ["Курский", "Октябрьский", "Большесолдатский", "Обоянский", "Пристенский", "Солнцевский"],
    "Обнинск": ["Боровский", "Малоярославецкий", "Жуковский"],
    "Прилузский": ["Вилегодский", "Лузский", "Опаринский", "Мурашинский", "Нагорский", "Койгородский",
                   "Сысольский", "Юрьянский", "Слободской"],
    "Пудожский": ["Медвежьегорский", "Онежский", "Плесецкий", "Каргопольский", "Вытегорский", "Подпорожский", "Прионежский"],
    "Смирныховский": ["Александровск-Сахалинский", "Тымовский", "Ногликский", "Поронайский", "Углегорский"],
    "Томский": ["Северск", "Томск", "Шегарский", "Кожевниковский", "Болотнинский", "Юргинский", "Яшкинский",
                "Тайгинский", "Яйский", "Зырянский", "Асиновский", "Кривошеиновский"],
    "Хасанский": ["Владивостокский", "Надеждинский", "Уссурийский"],
    "Чекмагушевский": ["Илишевский", "Бакалинский", "Шаранский", "Будзякский", "Благоварский", "Кушнаренковский", "Дюртюлинский"]
}


excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t6+.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет подходящих муниципалитетов событий
В регионе Магаданская нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,ПотреблениеЭ_protest,ПотреблениеВ_protest,Бюджет_protest,...,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,Квартиры_event,mahalanobis_distance
0,Тверская,Андреапольский,28602000,20.4,0.00,0.0,400.0,953.00,5.30,56587.0,...,0.00,4600.0,330.0,993.30,0.00,19706.5,3.48,167100.0,636.54,0.586954
1,Тверская,Селижаровский,28650000,24.2,0.00,100.0,1320.0,1328.00,0.00,51706.0,...,0.00,0.0,0.0,1582.00,0.00,-5133.0,3.30,163299.0,872.81,0.979061
2,Тверская,Тверь,28701000,76.1,0.00,0.0,300.0,692.30,34.90,-427870.0,...,0.00,403.0,268.0,508.25,33.24,-89125.0,0.00,5852.0,414.08,3.885367
3,Тверская,Торопецкий,28655000,29.1,0.00,0.0,1400.0,702.00,0.00,-1776.0,...,0.00,0.0,0.0,728.00,0.00,1992.0,3.00,96222.0,0.00,0.324949
4,Тверская,Торжокский,28654000,4.8,0.00,0.0,4260.0,684.00,37.30,8640.0,...,0.00,0.0,0.0,626.82,41.80,0.0,0.00,4629.3,0.00,0.492911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Костромская,Солигаличский,34640000,0.0,28.95,50.0,349.0,590.00,0.00,1236.0,...,0.00,0.0,4100.0,596.00,0.00,345.0,0.00,339783.0,0.00,0.043675
153,Липецкая,Липецк,42701000,71.8,53.87,300.0,1600.0,967.56,15.52,130543.0,...,64.72,0.0,390.0,732.55,10.02,50188.3,0.50,150537.0,0.00,1.148946
154,Омская,Омск,52701000,211.7,49.50,600.0,4100.0,1009.00,29.00,-587202.0,...,32.83,45.0,1181.0,869.31,11.18,-115534.0,0.20,359072.0,0.00,5.483534
155,Псковская,Псков,58701000,0.0,20.60,300.0,5670.0,749.20,29.00,-44726.7,...,86.50,100.0,400.0,637.00,22.71,-61944.0,0.00,5699.0,0.00,0.472147


In [27]:
df_2012 = pd.read_csv('прототип_данных_2010t6+.csv')
df_2011 = pd.read_csv('без_протестов2008_2010t6+.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)


merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
]]


from2012.to_csv('связка_махаланобис_2010t6+.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event
0,Торопецкий,28655000,29.1,0.00,0.0,1400.0,Кесовогорский,28626000,44.2,0.00,0.0,0.0
1,Торжокский,28654000,4.8,0.00,0.0,4260.0,Кимры,28726000,0.0,0.00,0.0,0.0
2,Холмогорский,11656000,90.0,0.00,620.0,1601.0,Шенкурский,11658000,0.0,2.39,0.0,0.0
3,Аургазинский,80605000,2.1,12.00,0.0,30100.0,Кугарчинский,80638000,0.3,0.00,0.0,0.0
4,Янаульский,80659000,20.0,14.20,0.0,5800.0,Уфа,80701000,0.0,79.10,0.0,5300.0
...,...,...,...,...,...,...,...,...,...,...,...,...
101,Нягань,71879000,211.7,88.52,0.0,1600.0,Покачи,71884000,6.6,0.00,0.0,0.0
102,Солигаличский,34640000,0.0,28.95,50.0,349.0,Вохомский,34606000,0.0,0.00,0.0,4100.0
103,Липецк,42701000,71.8,53.87,300.0,1600.0,Задонский,42624000,0.0,64.72,0.0,390.0
104,Псков,58701000,0.0,20.60,300.0,5670.0,Великие,58710000,0.0,86.50,100.0,400.0


In [ ]:
df = pd.read_csv('общие_данные_2010t6+.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t6+.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t6+.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event
0,2008,Торопецкий,28655000,24.0,0.00,300.0,500.0,Тверская,Кесовогорский,28626000.0,32.1,0.00,0.0,0.0,Тверская
1,2009,Торопецкий,28655000,29.0,0.00,0.0,0.0,Тверская,Кесовогорский,28626000.0,46.4,0.00,0.0,0.0,Тверская
2,2010,Торопецкий,28655000,29.1,0.00,0.0,0.0,Тверская,Кесовогорский,28626000.0,47.6,0.00,0.0,0.0,Тверская
3,2011,Торопецкий,28655000,29.1,0.00,0.0,500.0,Тверская,Кесовогорский,28626000.0,47.5,0.00,0.0,536.0,Тверская
4,2012,Торопецкий,28655000,29.1,0.00,0.0,1400.0,Тверская,Кесовогорский,28626000.0,47.2,0.00,0.0,0.0,Тверская
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2016,Томский,69654000,0.0,61.59,100.0,9900.0,Томская,Чаинский,69656000.0,0.0,23.81,0.0,2426.0,Томская
1351,2017,Томский,69654000,0.0,61.38,100.0,8600.0,Томская,Чаинский,69656000.0,0.0,33.37,0.0,607.0,Томская
1352,2018,Томский,69654000,0.0,61.18,100.0,12390.0,Томская,Чаинский,69656000.0,0.0,46.88,0.0,186.0,Томская
1353,2019,Томский,69654000,0.0,58.65,50.0,9410.0,Томская,Чаинский,69656000.0,0.0,50.49,0.0,30.0,Томская
